# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


Ambiente locale rilevato.


# Feature Extractor Setup

In [ ]:
# --- CELLA 2: DATI RAW ---
# Percorsi (Adattali se il nome dello zip su Drive è diverso)
ZIP_VIDEOS = '/content/drive/MyDrive/AML_MistakeDetection_DATA/videos.zip'
LOCAL_VIDEO_DIR = '/content/data/raw_videos'
DATA_ROOT = '/content/data'

# Estrazione Video
if IS_COLAB and not os.path.exists(LOCAL_VIDEO_DIR):
    print(f"Estrazione video da {ZIP_VIDEOS}...")
    !mkdir -p {LOCAL_VIDEO_DIR}
    !unzip -q "{ZIP_VIDEOS}" -d "{LOCAL_VIDEO_DIR}"
    print("✅ Video estratti e pronti.")
else:
    print("✅ Video già presenti o ambiente locale.")

In [ ]:
# --- CELLA 3: ESTRAZIONE ---
# Importa le classi dal tuo file nel repo
from utils.feature_extractor import PerceptionFeatureExtractor, ExtractionConfig

# 1. Configurazione
config = ExtractionConfig(
    video_dir = LOCAL_VIDEO_DIR,   # Cartella video scompattati
    output_root = DATA_ROOT,       # Dove salvare (creerà /perception_encoder/segment/1s)
    model_name = 'vit_base_patch16_224.fb_pe_core', 
    batch_size = 64                # Riduci a 32 se la GPU va in errore (OOM)
)

# 2. Esecuzione
print("Avvio Perception Encoder Feature Extraction...")
extractor = PerceptionFeatureExtractor(config)
extractor.run()

In [ ]:
# --- CELLA 4: SALVATAGGIO SU DRIVE ---
OUTPUT_ZIP_NAME = 'features_perception_encoder.zip'
OUTPUT_PATH_DRIVE = os.path.join('/content/drive/MyDrive/AML_MistakeDetection_DATA', OUTPUT_ZIP_NAME)

print(f"Compressione e upload su Drive: {OUTPUT_PATH_DRIVE}")

# Ci spostiamo nella root dei dati per zippare la cartella corretta
%cd {DATA_ROOT}

# Zippa ricorsivamente la cartella generata dall'extractor
!zip -r -q "{OUTPUT_PATH_DRIVE}" perception_encoder/

print("✅ Fatto! Zip salvato su Drive. Ora puoi usarlo per il training.")